In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np 

# Vectorization and Broadcasting

Vectorization is the absence of explicit loop during the development of the code 

In [ ]:
np.random.randint(1, 10, (1, 2)) * np.random.randint(1, 10, (2, 1))

X = lambda x, y: np.random.randint(1, 10, (x, y))
X(1, 2) * X(2, 1)
np.dot(X(1, 2), X(2,1))
X(1, 2) * X(3, 1)
#np.dot(X(1, 2), X(3, 1))

Broadcasting allows an operator or a function to ...

In [ ]:
A = np.arange(16).reshape(4,4)
b = np.arange(4)
A
b 

In [ ]:
A + b  

In [ ]:
m = np.arange(6).reshape(3, 1, 2)
n = np.arange(6).reshape(3, 2, 1)
m
n 

In [ ]:
m + n

In [ ]:
?np.random.randint

In [ ]:
type(np.arange(10))

# Array vs list

* Faster in time and less memory space
    * datatype of every element in the array is identical
* convinent in usage

<span style="font-family:New York Times; font-size:1em; color:red;">


Explanation needed here! 

In [ ]:
import sys 

[`sys.getsizeof()`](https://docs.python.org/3/library/sys.html#sys.getsizeof)

> Return the size of an object in bytes. The object can be any type of object.

> Only the memory consumption directly attributed to the object is accounted for, not the memory consumption of objects it refers to.

> If given, default will be returned if the object does not provide means to retrieve the size. Otherwise a TypeError will be raised.

> getsizeof() calls the object’s __sizeof__ method and adds an additional garbage collector overhead if the object is managed by the garbage collector.

In [ ]:
# By itself, the builtin function sys.getsizeof() is not helpful 
# determining the size of a container and all of its contents

from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)


##### Example call #####
if __name__ == '__main__':
    d = dict(a=1, b=2, c=3, d=[4,5,6,7], e='a string of chars')
    print(total_size(d, verbose=True))

In [ ]:
x = (int, float, bool, list, set, tuple, dict)
n = len(x)
type(x)
x = x.__iter__()
i = 0
while i < n:
    sys.getsizeof(next(x))
    i += 1

In [ ]:
A = np.random.randint(1, 10, (20, 30))
sys.getsizeof(A)
sys.getsizeof(list(A))

B = list(range(10))
sys.getsizeof(B)
sys.getsizeof(np.array(B))

In [ ]:
A = np.random.randint(1, 10, (20, 30))
A.__len__()
A.shape

In [ ]:
AList = [[3, 5, 7], [1, 7, 5]]

In [ ]:
sys.getsizeof(AList)

when new item is added to a list, two things happen
* The extra item fit into spare space.(It is a seems a waste when the list is small)
* No extra space, so a new list is created and the content copied across, then extra item is added. (It cost too much)

Enough memory is allocated (to save memory in these common cases)

In [ ]:
sys.getsizeof([])
sys.getsizeof([1])
sys.getsizeof(2)
sys.getsizeof(np.array([1]))
sys.getsizeof([1, 2])
sys.getsizeof((list(range(1))))
sys.getsizeof([[1],[]])
sys.getsizeof([[1],[2]])

## Incresement pattern of the size of array 

![](https://www.laurentluce.com/images/blog/list/list_insert.png)

# `np.sin` vs `math.sin`

https://stackoverflow.com/questions/57124500/numpy-create-sine-wave-with-exponential-decay/57124613#comment100767313_57124613

Calculate 
$$\sin(n) \cdot 2^{-n\cdot\mathrm{factor}}$$

In [ ]:
import numexpr as ne

> `NumExpr` is a fast numerical expression evaluator for NumPy. With it, expressions that operate on arrays (like `3*a+4*b`) are accelerated and use less memory than doing the same calculation in Python.

In [ ]:
def orig(n_max, factor):
    n = np.arange(n_max)
    return np.sin(n) * 2**(-n * factor)


#Rory Daulton's solution
def mod(n_max, factor):
    n = np.arange(n_max)
    newfactor = -np.log(2) * factor
    return np.sin(n) * np.exp(newfactor * n)


def mod_2(n_max, factor):
    n = np.arange(n_max)
    return ne.evaluate("sin(n) * 2**(-n * factor)")


#Rory Daulton's solution using Numexpr
def mod_3(n_max, factor):
    n = np.arange(n_max)
    newfactor = -np.log(2) * factor
    return ne.evaluate("sin(n) * exp(newfactor * n)")

In [ ]:
_ = orig(1e6, 0.5)

In [ ]:
%timeit _=orig(1e6, 0.5)

In [ ]:
%timeit _ = mod(1e5, 0.5)

In [ ]:
%timeit _ =  mod_2(1e5, 0.5)

In [ ]:
%timeit _ = mod_3(1e5, 0.5)

## vectorized function

In [ ]:
x = np.linspace(-1, 2, 1e7)
y = np.sin(np.pi * x)
y 

In [ ]:
x = np.linspace(-1, 2, 1e7)
pi = np.pi
y = ne.evaluate("sin(pi * x)")
y 

In [ ]:
import math

In [ ]:
x = np.linspace(-1, 2, 1e7)
_ = [math.sin(math.pi * i) for i in x]